In [20]:
import re
import pickle
from tqdm import tqdm
from sklearn.utils import shuffle
import numpy as np
from tqdm import tqdm
import bz2
from keras.layers import *
from keras.models import Model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [2]:
def splitReviewsLabels(lines):
    reviews = []
    labels = []
    for review in tqdm(lines):
        rev = reviewToX(review)
        label = reviewToY(review)
        reviews.append(rev[:512])
        labels.append(label)
    return reviews, labels

In [3]:
def reviewToY(review):
    return [1,0] if review.split(' ')[0] == '__label__1' else [0,1] 

In [4]:
def reviewToX(review):
    review = review.split(' ', 1)[1][:-1].lower()
    review = re.sub('\d','0',review)
    if 'www.' in review or 'http:' in review or 'https:' in review or '.com' in review:
        review = re.sub(r"([^ ]+(?<=\.[a-z]{3}))", "<url>", review)
    return review

In [5]:
train_file = bz2.BZ2File('data/train.ft.txt.bz2')
test_file = bz2.BZ2File('data/test.ft.txt.bz2')

In [6]:
train_lines = train_file.readlines()
test_lines = test_file.readlines()

In [7]:
train_lines = [x.decode('utf-8') for x in train_lines]
test_lines = [x.decode('utf-8') for x in test_lines]

In [8]:
# Load from the file
reviews_train, y_train = splitReviewsLabels(train_lines)
reviews_test, y_test = splitReviewsLabels(test_lines)

100%|██████████| 400000/400000 [00:05<00:00, 74171.77it/s]


In [9]:
y_train = np.array(y_train)
y_test = np.array(y_test)

In [10]:
max_features = 8192
maxlen = 128
embed_size = 64

In [11]:
tokenizer = Tokenizer(num_words=max_features)

In [12]:
tokenizer.fit_on_texts(reviews_train)

In [13]:
token_train = tokenizer.texts_to_sequences(reviews_train)
token_test = tokenizer.texts_to_sequences(reviews_test)

In [14]:
x_train = pad_sequences(token_train, maxlen=maxlen, padding='post')
x_test = pad_sequences(token_test, maxlen=maxlen, padding='post')

In [15]:
input = Input(shape=(maxlen,))
net = Embedding(max_features, embed_size)(input)
net = Dropout(0.2)(net)
net = BatchNormalization()(net)

net = Conv1D(32, 7, padding='same', activation='relu')(net)
net = BatchNormalization()(net)
net = Conv1D(32, 3, padding='same', activation='relu')(net)
net = BatchNormalization()(net)
net = Conv1D(32, 3, padding='same', activation='relu')(net)
net = BatchNormalization()(net)
net = Conv1D(32, 3, padding='same', activation='relu')(net)
net1 = BatchNormalization()(net)

net = Conv1D(2, 1)(net)
net = GlobalAveragePooling1D()(net)
output = Activation('softmax')(net)
model = Model(inputs = input, outputs = output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 128, 64)           524288    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128, 64)           0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 128, 64)           256       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 128, 32)           14368     
_________________________________________________________________
batch_normalization_2 (Batch (None, 128, 32)           128       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 128, 32)           3104      
__________

In [16]:
model.fit(x_train, y_train, batch_size=2048, epochs=5, validation_split=0.1)

Train on 3240000 samples, validate on 360000 samples
Epoch 1/5
3240000/3240000 [==============================] - 177s 55us/step - loss: 0.1935 - acc: 0.9237 - val_loss: 0.1583 - val_acc: 0.9414
Epoch 2/5
3240000/3240000 [==============================] - 172s 53us/step - loss: 0.1569 - acc: 0.9414 - val_loss: 0.1537 - val_acc: 0.9442
Epoch 3/5
3240000/3240000 [==============================] - 165s 51us/step - loss: 0.1477 - acc: 0.9451 - val_loss: 0.1604 - val_acc: 0.9400
Epoch 4/5
3240000/3240000 [==============================] - 164s 51us/step - loss: 0.1417 - acc: 0.9477 - val_loss: 0.1484 - val_acc: 0.9460
Epoch 5/5
3240000/3240000 [==============================] - 165s 51us/step - loss: 0.1372 - acc: 0.9494 - val_loss: 0.1684 - val_acc: 0.9363


In [17]:
model.evaluate (x_test, y_test)

400000/400000 [==============================] - 36s 90us/step


[0.1724140334594995, 0.9349575]

In [18]:
# serialize model to JSON
model_json = model.to_json()
with open("data/model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("data/model.h5")
print("Saved model to disk")

Saved model to disk


In [21]:
# saving
with open('data/tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)